# Попытка улучшить показатели лучшей реализации с ошибкой в ~12.93% RMSE
Тут пробуем отказаться от всех нормализаций данных, передав в catboost все as is.

In [1]:
from IPython.lib.deepreload import reload
from xgboost import train
%load_ext autoreload
%autoreload 2

import joblib
import numpy as np
import pandas as pd

from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, root_mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score

from utils.data_manager import DataManager
from utils.model_manager import ModelManager

## Загрузка данных

In [2]:
data_path = 'data/home-data-for-ml-course'
train_data = pd.read_csv(data_path + '/train.csv')
test_data = pd.read_csv(data_path + '/test.csv')
train_data.shape, test_data.shape

((1460, 81), (1459, 80))

## Предобработка данных и построение модели через Catboost чистый

In [3]:
dm = DataManager()

In [4]:
intuitively_bad_features = [
    'LotShape',  # Общая форма участка
    'LandContour',  # Рельеф участка
    'LotConfig',  # Конфигурация участка
    'LandSlope',  # Уклон участка
    'MiscFeature',
    'MiscVal',
]
bad_columns = dm.get_all_nan_cols(train_data)
bad_columns.append('Id')
bad_columns.extend(intuitively_bad_features)
bad_columns

['Id',
 'LotShape',
 'LandContour',
 'LotConfig',
 'LandSlope',
 'MiscFeature',
 'MiscVal']

In [5]:
train_data = train_data.drop(columns=bad_columns)
train_data.shape

(1460, 74)

In [6]:
X, y = dm.split_data_set_to_x_y(train_data, 'SalePrice')
print(X.shape, y.shape)

(1460, 73) (1460,)


In [7]:
# Получение числовых колонок
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns
# Получение нечисловых колонок (всех остальных)
non_numeric_columns = X.select_dtypes(exclude=['float64', 'int64']).columns

In [8]:
RANDOM_STATE = 42
# 1. Откладываем 20% данных на итоговый тест
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

# 2. Разбиваем оставшиеся 80% на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=RANDOM_STATE)

# Проверка размера полученных наборов
print(f'Train size: {len(X_train)}')
print(f'Validation size: {len(X_val)}')
print(f'Test size: {len(X_test)}')

Train size: 876
Validation size: 292
Test size: 292


In [9]:
# Заполнение пропусков только в категориальных признаках - так делаем, если не используем пайплайн sklearn 
for col in non_numeric_columns:
    X_train[col] = X_train[col].fillna('unknown')
    X_val[col] = X_val[col].fillna('unknown')
    X_test[col] = X_test[col].fillna('unknown')

In [10]:
# Инициализация и обучение CatBoost
cat_model = CatBoostRegressor(
    iterations=5000,
    learning_rate=0.05,
    depth=6,
    loss_function='RMSE',
    verbose=100,
    random_seed=RANDOM_STATE,
    # early_stopping_rounds=100  # останавливаем обучение через 20 итераций после детекции оптимального значения
    # l2_leaf_reg=1,  # можно уменьшить до 1-2, чтобы модель лучше учила данные
    # border_count=256,  # увеличение до 128 или даже 256 помогает точнее делить числовые признаки
    # random_strength=0.5,  # уменьшение до 1 (или даже 0.5) позволяет модели точнее подстраиваться под данные
    # bagging_temperature=0.3  # можно немного снизить, чтобы увеличить стабильность
    
    # ctr_target_border_count=20,
    # ctr_leaf_count_limit=50,
    # store_all_simple_ctr=True  # это сохранит больше промежуточных значений CTR, помогая качеству
)

cat_model.fit(
    X_train, y_train,
    cat_features=list(non_numeric_columns),
    eval_set=(X_val, y_val),
    use_best_model=True,
)

0:	learn: 74721.3226634	test: 74929.3601726	best: 74929.3601726 (0)	total: 156ms	remaining: 12m 58s
100:	learn: 24517.3897780	test: 30471.0993737	best: 30471.0993737 (100)	total: 6.92s	remaining: 5m 35s
200:	learn: 17548.1023597	test: 27037.1442010	best: 27037.1442010 (200)	total: 13.8s	remaining: 5m 28s
300:	learn: 14789.6595772	test: 26020.8456591	best: 26020.8456591 (300)	total: 21.1s	remaining: 5m 28s
400:	learn: 13171.3878976	test: 25625.4933304	best: 25607.6615998 (381)	total: 28.5s	remaining: 5m 26s
500:	learn: 12088.4435666	test: 25340.8346331	best: 25340.8346331 (500)	total: 36s	remaining: 5m 23s
600:	learn: 11104.7157298	test: 25160.7488835	best: 25147.3695141 (570)	total: 43.2s	remaining: 5m 16s
700:	learn: 10055.0164308	test: 24958.2354686	best: 24958.2354686 (700)	total: 50.7s	remaining: 5m 10s
800:	learn: 9172.3179638	test: 24847.4695204	best: 24845.6962560 (793)	total: 58.3s	remaining: 5m 5s
900:	learn: 8486.4359814	test: 24770.2599407	best: 24760.3757210 (887)	total: 1m

In [37]:
# Финальное предсказание и оценка на тестовых данных (RMSE)
y_pred_test = cat_model.predict(X_test)
rmse_test = root_mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)
mean_target = np.mean(y_test)

print(f'Финальный R2 на тестовом наборе: {r2_test:.4f}')
print(f'Финальный RMSE на тестовом наборе: {rmse_test:.2f}')
print(f'Относительная ошибка (RMSE/mean): {rmse_test/mean_target*100:.2f}%')

NameError: name 'cat_model' is not defined

# Итого
Передача в катбуст чистых данных не привела ни к чему хорошему. Показатели ошибки вернулись до уровня 15+%. Пробовал передавать нормализованные числовые данные - безуспешно. Ни на что не влияет. Таким образом, внутренний механизм работы с категориальными признаками катбуста ничем не лучше onehot дефолтной нормализации.